# 1. Household Water Consumption Data Generator

In [1]:
# importing packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import random

In [2]:
periods, divider = 300, 6

# Example 1: Creating a DatetimeIndex with a frequency
start_date = '2023-01-01'
index_freq = '1H'  # Daily frequency

# Create DateTimeIndex at minute frequency
def household_data_generator():
    index = pd.date_range(start=start_date, periods=periods, freq=index_freq)
    data = np.array([i+random.randint(1,4) for i in range(int(periods/divider)) for j in range(divider)])
    df = pd.DataFrame(data, index=index, columns=['Value'])
    return df
df = household_data_generator()
df

,Value
2023-01-01 00:00:00,4
2023-01-01 01:00:00,1
2023-01-01 02:00:00,1
2023-01-01 03:00:00,3
2023-01-01 04:00:00,2
...,...
2023-01-13 07:00:00,52
2023-01-13 08:00:00,50
2023-01-13 09:00:00,53
2023-01-13 10:00:00,50


# 2. Understanding General, Seasonal and Event-based Trends

In [3]:
!pip install pmdarima
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf,  plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.1 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split 
train_df, test_df = train_test_split(df, test_size = 0.2, shuffle=False)
# Perform additive seasonal decomposition
decomposition = sm.tsa.seasonal_decompose(df, model='additive')

# Access the decomposed components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

fig = go.Figure()
fig.add_trace(go.Scatter(x=seasonal.index, y=seasonal, name='Daily Trends'))
fig.update_layout(
    height=600,
    width=800,
    showlegend=True,
    title='Daily Trends Plot'
)
fig.show()

# 3. Optimization Hyper-parameter using SARIMAX

In [5]:
from statsmodels.tsa.arima.model import ARIMA

In [6]:
import itertools
limit = 3
pdq = list(itertools.product(range(0, limit), range(1, limit), range(0, limit)))

aic, number = [],[]
number = []
for i in pdq:
    # Training the model
    model = ARIMA(train_df.values, order=(i))
    model_fit = model.fit()
    # Consolidate training label and metrics
    print(f'ARIMA({i}) AIC : {round(model_fit.aic,2)}')
    aic.append(round(model_fit.aic, 2))
    number.append(i)
    
model = ARIMA(train_df.values, order=number[aic.index(min(aic))])
model_fit = model.fit()
prediction = model_fit.forecast(len(test_df))
prediction_value = prediction
prediction_index = list(test_df.index)

ARIMA((0, 1, 0)) AIC : 866.04
ARIMA((0, 1, 1)) AIC : 796.93
ARIMA((0, 1, 2)) AIC : 798.89
ARIMA((0, 2, 0)) AIC : 1104.85
ARIMA((0, 2, 1)) AIC : 868.13


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.



ARIMA((0, 2, 2)) AIC : 732.39
ARIMA((1, 1, 0)) AIC : 828.28
ARIMA((1, 1, 1)) AIC : 798.9
ARIMA((1, 1, 2)) AIC : 797.82
ARIMA((1, 2, 0)) AIC : 1002.63
ARIMA((1, 2, 1)) AIC : 827.04


/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



ARIMA((1, 2, 2)) AIC : 733.55
ARIMA((2, 1, 0)) AIC : 803.73
ARIMA((2, 1, 1)) AIC : 797.28
ARIMA((2, 1, 2)) AIC : 739.41
ARIMA((2, 2, 0)) AIC : 929.63
ARIMA((2, 2, 1)) AIC : 795.49
ARIMA((2, 2, 2)) AIC : 732.66


In [7]:
number[aic.index(min(aic))]

(0, 2, 2)

In [8]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  240
Model:                 ARIMA(0, 2, 2)   Log Likelihood                -363.195
Date:                Sat, 22 Jul 2023   AIC                            732.390
Time:                        16:02:46   BIC                            742.807
Sample:                             0   HQIC                           736.589
                                - 240                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -1.9978      2.545     -0.785      0.433      -6.987       2.991
ma.L2          0.9978      2.539      0.393      0.694      -3.979       5.974
sigma2         1.1452      2.927      0.391      0.696      -4.591       6.882
===================================================================================
Ljung-Box (L1) (Q):                   1.28   Jarque-Bera (JB):                 9.41
Prob(Q):                              0.26   Prob(JB):                         0.01
Heteroskedasticity (H):               1.24   Skew:                            -0.08
Prob(H) (two-sided):                  0.34   Kurtosis:                         2.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [9]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add traces to the figure
fig.add_trace(go.Scatter(x=train_df.index, y=train_df['Value'], name='Train Data'))
fig.add_trace(go.Scatter(x=prediction_index, y=prediction_value, name='Prediction', line=dict(color='red')))
fig.add_trace(go.Scatter(x=test_df.index, y=test_df['Value'], name='Real Visit Data', line=dict(color='orange', dash='dash')))

# Update the layout
fig.update_layout(
    title='Water Consumption Projections',
    xaxis_title='Date',
    yaxis_title='Value',
    legend=dict(x=0, y=1, traceorder='normal'),
    width=800,
    height=400)

# Show the plot
fig.show()

# 4. End-to-end ARIMA Algorithm (improved)

In [10]:
def ARIMA_training(train_df, test_df ,order=(0, 2, 2)):
    train_df, test_df = train_test_split(df, test_size = 0.2, shuffle=False)
    model = ARIMA(train_df.values, order=order)
    model_fit = model.fit()
    prediction = model_fit.forecast(len(test_df))
    prediction_value = prediction
    prediction_index = list(test_df.index)
    print(model_fit.summary())
    return [prediction_index, prediction_value]
def plotly_graph(train_df, test_df, prediction_index, prediction_value):
    import plotly.graph_objects as go

    # Create the figure
    fig = go.Figure()

    # Add traces to the figure
    fig.add_trace(go.Scatter(x=train_df.index, y=train_df['Value'], name='Train Data'))
    fig.add_trace(go.Scatter(x=prediction_index, y=prediction_value, name='Prediction', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=test_df.index, y=test_df['Value'], name='Real Visit Data', line=dict(color='orange', dash='dash')))

    # Update the layout
    fig.update_layout(
        title='Water Consumption Projections',
        xaxis_title='Date',
        yaxis_title='Value',
        legend=dict(x=0, y=1, traceorder='normal'),
        width=800,
        height=400)

    # Show the plot
    fig.show()
def prediction_gradient(prediction_value):
    # Create X and y for linear regression
    X = np.arange(len(prediction_value)).reshape(-1, 1)
    y = prediction_value

    # Perform linear regression
    coefficients = np.polyfit(X.squeeze(), y, 1)
    gradient, intercept = coefficients[0], coefficients[1]
    # Calculate the predicted values y_pred for R-squared score
    y_pred = gradient * X.squeeze() + intercept
    r2 = r2_score(y, y_pred)
    
    # return gradient, intercept
    return [gradient, intercept, r2]
def main(df):
    train_df, test_df = train_test_split(df, test_size = 0.2, shuffle=False)
    [prediction_index, prediction_value] = ARIMA_training(train_df, test_df ,order=(0, 2, 2))
    plotly_graph(train_df, test_df, prediction_index, prediction_value)
    print(prediction_gradient(prediction_value))
main(df)

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning:

Non-invertible starting MA parameters found. Using zeros as starting parameters.



                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  240
Model:                 ARIMA(0, 2, 2)   Log Likelihood                -363.195
Date:                Sat, 22 Jul 2023   AIC                            732.390
Time:                        16:02:47   BIC                            742.807
Sample:                             0   HQIC                           736.589
                                - 240                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -1.9978      2.545     -0.785      0.433      -6.987       2.991
ma.L2          0.9978      2.539      0.393      0.694      -3.979       5.974
sigma2         1.1452      2.927      0.391      0.6

NameError: name 'r2_score' is not defined

# 5. Populating Household Datasets towards Benchmarking

In [ ]:
from functools import reduce
data_frames = [household_data_generator() for i in range(10)]
merged_df = reduce(lambda left, right: pd.merge(left, 
                                                right, left_index=True, right_index=True), data_frames)
merged_df['Value'] = merged_df.sum(axis=1)
merged_df


In [ ]:
main(merged_df[['Value']])